In [123]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

### LECTURA DATOS

In [124]:
#mysql_config = {
#    'host': 'localhost',
#    'user': 'root',
#    'database': 'cancer_mama'
#}

#conn_mysql = mysql.connector.connect(**mysql_config)
#cursor_mysql = conn_mysql.cursor()

#query = "SELECT * FROM tumor;"
#df_sql = pd.read_sql_query(query, conn_mysql)

In [125]:
# PARA MAC
# df_sql = pd.read_csv('./Datos/datos_sql.csv')
# df_sql = df_sql.drop('Unnamed: 0',axis=1)
# df_sql

In [126]:
df_csv1 = pd.read_csv('./Datos/patients_batch1.csv')
df_csv2 = pd.read_csv('./Datos/patients_batch2.csv')
df_excel = pd.read_excel('./Datos/gynecological.xlsx')
df_excel = df_excel.drop('Unnamed: 0', axis=1)
df_json = pd.read_json('./Datos/histochemistry.json')
df_sql = pd.read_csv('./Datos/datos_sql.csv')
df_sql = df_sql.drop('Unnamed: 0', axis=1)
df_sql


,ehr,n_tumor,t_category,n_category,m_category,t_category_after_neoadj,n_category_after_neoadj,m_category_after_neoadj,stage_diagnosis,stage_after_neo,grade,ductal,lobular,neoadjuvant
0,10011773,1,IS,0,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,no
1,10020495,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,1.0,NaN,no
2,10020495,2,3,1,0,2,0.0,NaN,IA,NaN,2,1.0,NaN,yes
3,10030299,1,1,0,0,NaN,NaN,NaN,IA,NaN,1,1.0,NaN,no
4,10030824,1,2,1,0,2,2.0,0.0,IIIA,IIIA,3,NaN,1.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11162,99966287,1,1,1,0,1,0.0,0.0,IB,IIA,3,1.0,NaN,yes
11163,99981192,1,1,0,0,NaN,NaN,NaN,IA,NaN,2,NaN,NaN,no
11164,99988958,1,0,2,0,1,1.0,0.0,IIIA,IIA,1,1.0,NaN,yes
11165,99992350,1,2,0,0,3,2.0,0.0,IIA,IIIA,2,1.0,NaN,yes


### EDA (EXPLORACIÓN VISUAL DE LOS DATOS)


In [127]:
df_excel

,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,10011773,-8.0,0.0,NaN,0.0,NaN,71.0
1,10030299,3.0,NaN,0.0,0.0,NaN,NaN
2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,10053435,2.0,NaN,0.0,1.0,21.0,74.0
4,10111454,0.0,-6.0,NaN,0.0,16.0,55.0
...,...,...,...,...,...,...,...
7681,99948591,9.0,2.0,-4.0,0.0,12.0,47.0
7682,99961100,2.0,2.0,0.0,0.0,19.0,52.0
7683,99981192,3.0,3.0,0.0,-5.0,11.0,NaN
7684,99992350,4.0,3.0,NaN,-6.0,11.0,55.0


In [128]:
df = df_excel

In [129]:
nulos = df.isnull()

num_nulos_por_columna = nulos.sum()

print("Número de valores nulos por columna:")
print(num_nulos_por_columna)


Número de valores nulos por columna:
ehr                 0
pregnancy        1273
birth            1269
caesarean        1555
abort            1451
menarche_age     2785
menopause_age    3200
dtype: int64


Imputamos los nulos de pregnancy, que es la suma de las otras 3 variables restantes, birth,caesarean y abort.

In [130]:
def imputar_pregnancy(row):
    if pd.isnull(row['pregnancy']):
        return row['birth'] + row['caesarean'] + row['abort']
    else:
        return row['pregnancy']

In [131]:
df['pregnancy'] = df.apply(imputar_pregnancy, axis=1)
df


,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,10011773,-8.0,0.0,NaN,0.0,NaN,71.0
1,10030299,3.0,NaN,0.0,0.0,NaN,NaN
2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,10053435,2.0,NaN,0.0,1.0,21.0,74.0
4,10111454,0.0,-6.0,NaN,0.0,16.0,55.0
...,...,...,...,...,...,...,...
7681,99948591,9.0,2.0,-4.0,0.0,12.0,47.0
7682,99961100,2.0,2.0,0.0,0.0,19.0,52.0
7683,99981192,3.0,3.0,0.0,-5.0,11.0,NaN
7684,99992350,4.0,3.0,NaN,-6.0,11.0,55.0


In [132]:
nulos = df.isnull()

num_nulos_por_columna = nulos.sum()

print("Número de valores nulos por columna:")
print(num_nulos_por_columna)


Número de valores nulos por columna:
ehr                 0
pregnancy         442
birth            1269
caesarean        1555
abort            1451
menarche_age     2785
menopause_age    3200
dtype: int64


In [133]:
# Imputar valores nulos en la columna "birth"
def imputar_birth(row):
    if pd.isnull(row['birth']):
        return row['pregnancy'] - (row['caesarean'] + row['abort'])
    else:
        return row['birth']

# Aplicar la función a cada fila del DataFrame
df['birth'] = df.apply(imputar_birth, axis=1)

df

,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,10011773,-8.0,0.0,NaN,0.0,NaN,71.0
1,10030299,3.0,3.0,0.0,0.0,NaN,NaN
2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,10053435,2.0,1.0,0.0,1.0,21.0,74.0
4,10111454,0.0,-6.0,NaN,0.0,16.0,55.0
...,...,...,...,...,...,...,...
7681,99948591,9.0,2.0,-4.0,0.0,12.0,47.0
7682,99961100,2.0,2.0,0.0,0.0,19.0,52.0
7683,99981192,3.0,3.0,0.0,-5.0,11.0,NaN
7684,99992350,4.0,3.0,NaN,-6.0,11.0,55.0


In [134]:
def imputar_caesarean(row):
    if pd.isnull(row['caesarean']):
        return row['pregnancy'] - (row['birth'] + row['abort'])
    else:
        return row['birth']

# Aplicar la función a cada fila del DataFrame
df['caesarean'] = df.apply(imputar_caesarean, axis=1)

df

,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,10011773,-8.0,0.0,-8.0,0.0,NaN,71.0
1,10030299,3.0,3.0,3.0,0.0,NaN,NaN
2,10030824,0.0,0.0,0.0,NaN,NaN,44.0
3,10053435,2.0,1.0,1.0,1.0,21.0,74.0
4,10111454,0.0,-6.0,6.0,0.0,16.0,55.0
...,...,...,...,...,...,...,...
7681,99948591,9.0,2.0,2.0,0.0,12.0,47.0
7682,99961100,2.0,2.0,2.0,0.0,19.0,52.0
7683,99981192,3.0,3.0,3.0,-5.0,11.0,NaN
7684,99992350,4.0,3.0,7.0,-6.0,11.0,55.0


In [135]:
def imputar_abort(row):
    if pd.isnull(row['abort']):
        return row['pregnancy'] - (row['caesarean'] + row['birth'])
    else:
        return row['abort']

# Aplicar la función a cada fila del DataFrame
df['abort'] = df.apply(imputar_abort, axis=1)

df

,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,10011773,-8.0,0.0,-8.0,0.0,NaN,71.0
1,10030299,3.0,3.0,3.0,0.0,NaN,NaN
2,10030824,0.0,0.0,0.0,0.0,NaN,44.0
3,10053435,2.0,1.0,1.0,1.0,21.0,74.0
4,10111454,0.0,-6.0,6.0,0.0,16.0,55.0
...,...,...,...,...,...,...,...
7681,99948591,9.0,2.0,2.0,0.0,12.0,47.0
7682,99961100,2.0,2.0,2.0,0.0,19.0,52.0
7683,99981192,3.0,3.0,3.0,-5.0,11.0,NaN
7684,99992350,4.0,3.0,7.0,-6.0,11.0,55.0


In [136]:
nulos = df.isnull()

num_nulos_por_columna = nulos.sum()

print("Número de valores nulos por columna:")
print(num_nulos_por_columna)

Número de valores nulos por columna:
ehr                 0
pregnancy         442
birth             447
caesarean         923
abort             670
menarche_age     2785
menopause_age    3200
dtype: int64


In [137]:
# Imputar valores negativos en "pregnancy" basándose en "birth" y "abort"
def imputar_pregnancy_negativo(row):
    if row['pregnancy'] < 0:
        return abs(row['birth'] + row['abort']+row['caesarean'])
    else:
        return row['pregnancy']

# Aplicar la función a cada fila del DataFrame
df['pregnancy'] = df.apply(imputar_pregnancy_negativo, axis=1)

# Mostrar el DataFrame después de la imputación
print("DataFrame después de imputar valores negativos en 'pregnancy':")
df

DataFrame después de imputar valores negativos en 'pregnancy':


,ehr,pregnancy,birth,caesarean,abort,menarche_age,menopause_age
0,10011773,8.0,0.0,-8.0,0.0,NaN,71.0
1,10030299,3.0,3.0,3.0,0.0,NaN,NaN
2,10030824,0.0,0.0,0.0,0.0,NaN,44.0
3,10053435,2.0,1.0,1.0,1.0,21.0,74.0
4,10111454,0.0,-6.0,6.0,0.0,16.0,55.0
...,...,...,...,...,...,...,...
7681,99948591,9.0,2.0,2.0,0.0,12.0,47.0
7682,99961100,2.0,2.0,2.0,0.0,19.0,52.0
7683,99981192,3.0,3.0,3.0,-5.0,11.0,NaN
7684,99992350,4.0,3.0,7.0,-6.0,11.0,55.0


# PREPROCESADO

### Preprocesado patients_batch1/patients_batch2

Para el preprocesado vamos a comenzar uniendo los dataframes de origen csv, ya que comparten las mismas columnas. Para poder realizar el preprocesado de ambos de una manera más sencilla

In [138]:
df_csv = pd.merge(df_csv1, df_csv2, on=['ehr','birth_date','diagnosis_date','death_date'], how='outer')
df_csv

,ehr,birth_date,diagnosis_date,death_date
0,10011773,1959-07-05,2015-04-02,NaN
1,10020495,1953-10-02,2017-12-04,NaN
2,10030299,1966-08-16,2019-06-27,NaN
3,10030824,1953-03-03,2018-09-07,NaN
4,10041592,1959-07-06,2018-11-19,NaN
...,...,...,...,...
10495,99966287,16/12/1978,02/08/2018,NaN
10496,99981192,10/04/1949,28/10/2017,NaN
10497,99988958,21/01/1966,27/05/2013,NaN
10498,99992350,04/09/1939,17/12/2018,NaN


In [ ]:
df_csv.info()

Observamos que en nuestro dataset de csv tenemos variables de tipo int64 y object; El identificador del paciente tendrá valor numérico, luego el resto de variables (birth_date, diagnosis_date, death_date) con formato de objeto.

Por lo tanto, primero vamos a pasar las fechas a tipo 'datetime'. Y una vez que los tenemos en este tipo, ponemos todas las fechas en el mismo formato

In [ ]:
# Convertir las columnas objeto al formato datetime
df_csv['birth_date'] = pd.to_datetime(df_csv['birth_date'])
df_csv['diagnosis_date'] = pd.to_datetime(df_csv['diagnosis_date'])
df_csv['death_date'] = pd.to_datetime(df_csv['death_date'])

# Cambiamos los formatos a '%d/%m/%Y'
df_csv['birth_date'] = df_csv['birth_date'].dt.strftime('%d-%m-%Y')
df_csv['diagnosis_date'] = df_csv['diagnosis_date'].dt.strftime('%d-%m-%Y')
df_csv['death_date'] = df_csv['death_date'].dt.strftime('%d-%m-%Y')

df_csv

Como podemos apreciar a primera vista, la columna de death_date posee muchos nulos. Vamos a realizar un recuento de valores para esa columna para ver cuantos pacientes han fallecido de nuestro conjunto de datos

In [ ]:
valores = df_csv['death_date'].count()
valores

In [ ]:
valores = df_csv['death_date'].isnull().sum()
valores

Vemos que  hay 1184 pacientes que han fallecido, y 9316 pacientes estables.

Una vez que tenemos todos los datos en el mismo formato. Vamos a comprobar que las fechas de los pacientes entran en los rangos correctos:
- Dia: [1:31]
- Mes: [1:12]

Empezamos comprobando que el rango de los días es el correcto:

In [ ]:
columnas = ['birth_date', 'diagnosis_date', 'death_date']

for i in columnas:
    # Especifico el rango
    rango_días = (
        pd.to_datetime(df_csv[i], format='%d-%m-%Y', errors='coerce').dt.day < 1
        | (pd.to_datetime(df_csv[i], format='%d-%m-%Y', errors='coerce').dt.day > 31)
    )

    filas_fuera_de_rango = df_csv[rango_días]

    if not filas_fuera_de_rango.empty:
        print(f"Filas con días fuera del rango en {i}:")
        print(filas_fuera_de_rango)
    else:
        print(f"Todos los días en {i} están dentro del rango.")

Comprobamos ahora el rango de los meses:

In [ ]:
columnas = ['birth_date', 'diagnosis_date', 'death_date']

for i in columnas:
    # Especifico el rango
    rango_mes = (
        pd.to_datetime(df_csv[i], format='%d-%m-%Y', errors='coerce').dt.month < 1
        | (pd.to_datetime(df_csv[i], format='%d-%m-%Y', errors='coerce').dt.month > 12)
    )

    filas_fuera_de_rango = df_csv[rango_mes]

    if not filas_fuera_de_rango.empty:
        print(f"Filas con días fuera del rango en {i}:")
        print(filas_fuera_de_rango)
    else:
        print(f"Todos los días en {i} están dentro del rango.")

Por lo tanto, podemos concluir que tenemos todos los datos en el mismo formato, y con los valores correctos. Es decir, no tenemos ninguna fecha mal puesta

### Preprocesado datos_sql.csv

In [139]:
df_sql.to_csv('datos_sql.csv',)

In [140]:
df_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11167 entries, 0 to 11166
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ehr                      11167 non-null  int64  
 1   n_tumor                  11167 non-null  int64  
 2   t_category               11167 non-null  object 
 3   n_category               11167 non-null  int64  
 4   m_category               11167 non-null  int64  
 5   t_category_after_neoadj  5677 non-null   object 
 6   n_category_after_neoadj  5677 non-null   float64
 7   m_category_after_neoadj  5627 non-null   float64
 8   stage_diagnosis          11167 non-null  object 
 9   stage_after_neo          6064 non-null   object 
 10  grade                    11167 non-null  int64  
 11  ductal                   6324 non-null   float64
 12  lobular                  1564 non-null   float64
 13  neoadjuvant              11167 non-null  object 
dtypes: float64(4), int64(5